In [1]:
from mongoengine import *
import os
import pandas as pd
import numpy as np
import re
from datetime import datetime

使用** mongoengine **  
[connect](http://docs.mongoengine.org/guide/connecting.html#guide-connecting)进行数据库连接

In [2]:
connect('typhoon')

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary())

## 1 读取文件

### 读取方式1：使用pandas

In [3]:
targetfilename=r"CH1956BST.txt"
# mac 16
# targetpath=r"/Users/casablanca/03project/typhoonSearchSys/demo_data"
# mac 15
# targetpath=r"/Users/liusihan/Documents/01project/TyphoonSearchSys/demo_data"
# aw15
targetpath=r"D:\01proj\typhoon\TyphoonSearchSys\demo_data"
fullname=os.path.join(targetpath,targetfilename)

In [4]:
fullname

'D:\\01proj\\typhoon\\TyphoonSearchSys\\demo_data\\CH1956BST.txt'

不使用pandas的read_table

In [5]:
with open(fullname,'rb') as f:
    data=pd.read_table(f,sep='\s+',encoding='utf-8',header=None,infer_datetime_format=False)
    print('读取成功')

读取成功


In [8]:
data.iloc[169]

0          66666
1              0
2             17
3              9
4              0
5              1
6              6
7           Vera
8    2.01107e+07
Name: 169, dtype: object

In [9]:
data.iloc[169][8]

20110729.0

In [10]:
data.iloc[170]

0    1956070500
1             2
2           120
3          1260
4          1000
5            20
6           NaN
7           NaN
8           NaN
Name: 170, dtype: object

In [19]:
data.iloc[170][0]

1956070500

In [22]:
str(data.iloc[170][0])[2:4]

'56'

In [25]:
#数字补齐
num=5
n='%02d'%num
n

'05'

----

In [11]:
class GeoTyphoonRealData(Document):
    '''
        支持geojson的存储至mongodb的model
    '''
    code=StringField(max_length=10)
    num=StringField()
    date=DateTimeField()
    bp=FloatField()
    wsm=FloatField()
    # 注意此处与django中的类型不同，django的类型为IntegerField，mongoengine为IntField！
    level=IntField()
    # latlon=models.ForeignKey(Point,on_delete=models.CASCADE)
    latlon=PointField()
    meta = {'collection': 'geotyphoonrealdata'}
    
    def __str__(self):
        return f'code:{self.code}|num:{self.num}|date:{self.date}|bp:{self.bp}|wsm:{self.wsm}|level:{self.level}|latlon:{self.latlon}'

In [14]:
def convert2Typhoon(obj,code,num):
    '''
        根据传入的series将其转为typhoob Model
    '''
    lat=float(f"{str(obj[2])[:-1]}.{str(obj[2])[-1:]}")
    lon=float(f"{str(obj[3])[:-1]}.{str(obj[3])[-1:]}")
    stamp_str=obj[0]
    stamp=datetime.strptime(str(stamp_str),'%Y%m%d%H')
    typhoon_temp=GeoTyphoonRealData(code=code,
                                date=stamp,
                                num=num,
                                bp=obj[4],
                                wsm=obj[5],
                                level=obj[1],
                                latlon=[lon,lat])
    return typhoon_temp

In [15]:
convert2Typhoon(data.iloc[170],'Vera','17')

<GeoTyphoonRealData: code:Vera|num:17|date:1956-07-05 00:00:00|bp:1000.0|wsm:20.0|level:2|latlon:[126.0, 12.0]>